In [1]:
from obstools.atacr import DayNoise, TFNoise, EventStream, StaNoise, utils
import obstools.atacr.plotting as atplot
from pathlib import Path
from obspy.core import read, Stream, Trace, AttribDict, UTCDateTime
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '/Users/charlesh/Documents/Codes/ATaCR')
import ObsQA
import datetime
from obspy.clients.fdsn import Client
import glob as g
import os
import argparse
import obspy
from obstools.scripts import comply_calculate, atacr_clean_spectra, atacr_correct_event, atacr_daily_spectra, atacr_download_data, atacr_download_event, atacr_transfer_functions
from stdb.scripts import query_fdsn_stdb

In [2]:
stations, stations_set = ObsQA.io.getstalist()
eventsfolder = '/Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp/ATaCR_Python/EVENTS'
catalog = pd.read_pickle(eventsfolder + '/event_catalog.pkl')
# display(catalog)

In [3]:
# # ----Events----
# # stas = ['G03A']
# # STEPS = [4,5,6,7] #Everything but the download steps - About 4min for six stations.
# # STEPS = [1,2,3]
# STEPS = [1,2]
# # cat = catalog.copy()
# cat = catalog[np.in1d(catalog.Network,'YO')].copy()
# # cat = cat[np.in1d(cat.Station,stas)]
# # cat = cat.iloc[3:]
# # cat = cat.to_frame().T
# display(cat)
# S = cat['Station'].tolist()
# N = cat['Network'].tolist()
# for ii in range(len(cat)):
#     S = cat['Station'].tolist()[ii]
#     N = cat['Network'].tolist()[ii]
#     staname = N + '.' + S
#     subfolder = staname + '/'
#     print('Station: ' + staname +  ' (' + str(ii+1) + ' of ' + str(len(cat)) + ')')
#     # =============# =============# =============
#     icatalog = cat[(cat.Network==N) & (cat.Station==S)].copy()
#     ObsQA.io.Run_ATaCR(icatalog, STEPS=STEPS)

In [4]:
# S= stations_cull['Station'].tolist()#station
# C='?H?' #channels
# Minmag = 6.3
# Maxmag = 6.7
# N = 'YO'
# # S = ['A06B','X01','X09','X06']
# EventStart = cat.Start.iloc[0].strftime("%Y-%m-%d, %H:%M:%S")
# EventEnd = cat.End.iloc[-1].strftime("%Y-%m-%d, %H:%M:%S")
# !query_fdsn_stdb -N "YO" -C "?H?" -S "A06B,X01,X06,X09" ./Data/sta_query> ./Data/Step_1_7_StationMeta_logfile.log
# !query_fdsn_stdb -N "7D" -C "?H?" -S "M08A" ./Data/sta_query> ./Data/Step_1_7_StationMeta_logfile.log
# !atacr_download_event --min-mag="6.3" --max-mag="6.7" --start="2014-04-03, 00:00:00" --end="2015-04-03, 23:59:59" --limit="6" ./Data/sta_query.pkl> ./Data/Step_2_7_EventDownload_logfile.log

In [5]:
# d = ObsQA.io.loadpickles('/Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp/ATaCR_Python/sta_query.pkl').Output[0]
# d

In [6]:
# ----Noise----
# stas = ['G03A']
# STEPS = [4,5,6,7] #Everything but the download steps - About 4min for six stations.
# STEPS = [1,2,3]
# STEPS = [1,4,5,6,7]
STEPS = [1,3]
cat = catalog[np.in1d(catalog.Network,'YO')].copy()
# cat = catalog[np.in1d(catalog.Station,'M08A')].copy()
# cat = catalog.copy()
# cat = cat[np.in1d(cat.Station,stas)]
# cat = cat.iloc[3:]
# cat = cat.to_frame().T
display(cat)
for ii in range(len(cat)):
    S = cat['Station'].tolist()[ii]
    N = cat['Network'].tolist()[ii]
    staname = N + '.' + S
    subfolder = staname + '/'
    print('----[' + staname + ']----' + ' (' + str(ii+1) + ' of ' + str(len(cat)) + ')')
    # =============# =============# =============
    icatalog = cat[(cat.Network==N) & (cat.Station==S)].copy()
    ObsQA.io.Run_ATaCR(icatalog, STEPS=STEPS,chan='HH')

,Station,Network,Latitude,Longitude,Experiment,Instrument_Design,Seismometer,Environment,Pressure_Gauge,Water_Depth_m,...,Deployment_Length_days,Good_Channels,n_events,Magnitude_mw,Origin,Metadata,Averaging,Events,Files,Depth_KM
26,A06B,YO,35.804501,-74.648804,ENAM,BG,Guralp CMG3T 120,North Atlantic,DPG,1325.0,...,362,True,4,"[6.7, 6.4, 6.6, 6.5]","[2015-02-16T23:06:28.080000Z, 2015-03-23T04:51...","[[[resource_id, event_type, event_type_certain...","[sta, sta, sta, sta]","[2015.047.23.06, 2015.082.04.51, 2014.093.01.5...","[2015.047.23.06.HHZ.SAC, 2015.082.04.51.HHZ.SA...","[10.8, 127.1, 10.6, 13.0]"
27,X01,YO,36.225201,-74.579300,ENAM,BG,Guralp CMG3T 120,North Atlantic,DPG,1329.0,...,362,True,4,"[6.7, 6.3, 6.4, 6.7]","[2015-02-16T23:06:28.080000Z, 2015-02-16T22:00...","[[[resource_id, event_type, event_type_certain...","[sta, sta, sta, sta]","[2015.047.23.06, 2015.047.22.00, 2015.082.04.5...","[2015.047.23.06.HHZ.SAC, 2015.047.22.00.HHZ.SA...","[10.8, 18.1, 127.1, 203.6]"
28,X06,YO,33.650700,-72.205704,ENAM,BG,Guralp CMG3T 120,North Atlantic,DPG,5154.0,...,361,True,4,"[6.7, 6.4, 6.4, 6.4]","[2015-02-16T23:06:28.080000Z, 2015-03-23T04:51...","[[[resource_id, event_type, event_type_certain...","[sta, sta, sta, sta]","[2015.047.23.06, 2015.082.04.51, 2015.089.08.4...","[2015.047.23.06.HHZ.SAC, 2015.082.04.51.HHZ.SA...","[10.8, 127.1, 15.8, 15.8]"
29,X09,YO,32.516899,-74.353600,ENAM,BG,Guralp CMG3T 120,North Atlantic,DPG,4726.0,...,359,True,4,"[6.7, 6.4, 6.4, 6.4]","[2015-02-16T23:06:28.080000Z, 2015-03-23T04:51...","[[[resource_id, event_type, event_type_certain...","[sta, sta, sta, sta]","[2015.047.23.06, 2015.082.04.51, 2015.089.08.4...","[2015.047.23.06.HHZ.SAC, 2015.082.04.51.HHZ.SA...","[10.8, 127.1, 15.8, 15.8]"


----[YO.A06B]---- (1 of 4)
Step 1/7 - BEGIN: Station Metadata
Station Query File Written to: /Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp/ATaCR_Python/sta_query.pkl
Step 1/7 - COMPLETE: Station Metadata
Step 3/7 - BEGIN: Download Day Data
----Begin Noise Download----
Station Query File Written to: /Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp/ATaCR_Python/sta_query.pkl
Step 3/7 - COMPLETE: Download Day Data
----[YO.X01]---- (2 of 4)
Step 1/7 - BEGIN: Station Metadata
Station Query File Written to: /Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp/ATaCR_Python/sta_query.pkl
Step 1/7 - COMPLETE: Station Metadata
Step 3/7 - BEGIN: Download Day Data
----Begin Noise Download----
Station Query File Written to: /Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp/ATaCR_Python/sta_query.pkl
Step 3/7 - COMPLETE: Download Day Data
----[YO.X06]---- (3 of 4)
Step 1/7 - BEGIN: Station Metadata
Station Query File Written to: /Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp/ATaCR_Python/sta_query.p